In [1]:
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep
import json
import re
from datasets import load_dataset, Dataset

/home/dev/therapybot/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
!wget -O web_content.csv 'https://docs.google.com/spreadsheet/ccc?key=1pXY3z48RnjoFnfBYj7Fg2oBcmzJnDwZ29GPuRg-ccWc&output=csv'
!wget -O video_transcripts.csv 'https://docs.google.com/spreadsheet/ccc?key=1N7JDn1Ez1BpGejur5HSFqrjOzT8z_3Opo84BEed55PY&output=csv'
!wget -O reddit.csv 'https://docs.google.com/spreadsheet/ccc?key=1h_H7UREvflYVcsePznZlUtF7NyttuW-7y9g66HRkMVA&output=csv'

--2023-09-10 05:55:59--  https://docs.google.com/spreadsheet/ccc?key=1pXY3z48RnjoFnfBYj7Fg2oBcmzJnDwZ29GPuRg-ccWc&output=csv
Resolving docs.google.com (docs.google.com)... 142.250.125.100, 142.250.125.101, 142.250.125.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.100|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://docs.google.com/spreadsheets/d/1pXY3z48RnjoFnfBYj7Fg2oBcmzJnDwZ29GPuRg-ccWc/export?format=csv [following]
--2023-09-10 05:55:59--  https://docs.google.com/spreadsheets/d/1pXY3z48RnjoFnfBYj7Fg2oBcmzJnDwZ29GPuRg-ccWc/export?format=csv
Reusing existing connection to docs.google.com:443.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-14-bs-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/d00rr4d3m7e5icp2h0t9kf146s/1694325360000/112922963459535230484/*/1pXY3z48RnjoFnfBYj7Fg2oBcmzJnDwZ29GPuRg-ccWc?format=csv [following]
--2023-09-10 05:56:00--  https://doc-1

# Uncomment the dataset you want to preprocess

In [2]:
# infile = "web_content.csv"
# outfile = "web_content.jsonl"

# df = pd.read_csv(infile)
# df = df.dropna()

In [62]:
infile = "video_transcripts.csv"
outfile = "video_transcripts.jsonl"

df = pd.read_csv(infile)
df = df.dropna()
df["identifiers"] = None
df["title"] = df["title"] + "\n" + df["description"]
_ = df.pop("description")

In [88]:
infile = "reddit.csv"
outfile = "reddit.jsonl"

df = pd.read_csv(infile)
df = df.dropna()

# df

with open(outfile, "w") as f:
    for row in df.itertuples():
        data = {"conversations": [
            {"from": "human", "value": row.prompt},
            {"from": "gpt", "value": row.resposne},
        ]}
        f.write(json.dumps(data) + "\n")

In [63]:
df["identifiers"] = None
df

,source,title,Unnamed: 3,transcript,identifiers
0,https://www.youtube.com/watch?v=0IqkVLEsC3E,"Therapy Recordings - ""Anna"", session 3, part 1...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: welcome to the curious mind podcast...,None
1,https://www.youtube.com/watch?v=uTj33gQ4G38,"Therapy Recordings - ""Anna"", session 1, part 2...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: welcome back to the curious mind po...,None
2,https://www.youtube.com/watch?v=zyc4on_c-58,"Therapy Recordings - ""Anna"", session 3, part 2...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: this is the curious mind podcast my...,None
3,https://www.youtube.com/watch?v=GpwgwK5GCEA,"Therapy Recordings - ""Anna"", session 2, part 1...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: welcome to the curious mind podcast...,None
4,https://www.youtube.com/watch?v=6bryrnVWGx8,"Therapy Recordings - ""Hanni"", session 1\nThis ...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: everyone my name is gabriel ellis i...,None
6,https://www.youtube.com/watch?v=7LD8iC4NqXM,Case study clinical example CBT: First session...,https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: hi lucy nice to meet you hi so i un...,None
7,https://www.youtube.com/watch?v=W3FqtEVcFFM,"Therapy Recordings - ""Anna"", session 2, part 2...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: welcome to the curus mind podcast a...,None
8,https://www.youtube.com/watch?v=My9Lmsh1Sv0,"Therapy Recordings - ""Hanni"", session 2\nHanni...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: so welcome to our second session wi...,None
9,https://www.youtube.com/watch?v=pQx4f2u9R6E,"Therapy Recordings - ""Hanni"", session 3\nIn th...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: so welcome to our third session wit...,None
10,https://www.youtube.com/watch?v=X47ODjcasUI,"Therapy Recordings - ""Anna"", Session 1, Part 1...",https://storage.googleapis.com/dara-c1b52.apps...,Speaker 0: today i start a series of three rec...,None


# A bunch of prompts to identify speakers and export conversations to a sharegpt format

In [64]:
def run_identifier(idx, transcript):
    messages = [
        {
            "role": "system",
            "content": """
You are an AI analysing transcripts. 
Extract only the relevant part of the conversation to contain a dialogue between 2 individuals: a counsellor and their counselee.
You must only use the following JSON format: 
{
    "introducer": <The identifier of the speaker who gave the introduction to the conversation, or null>,
    "counsellor": <The identifier of the speaker that you extracted to be the counsellor>,
    "counselee": <The identifier of the speaker that you extracted to be the person receiving advice from the counsellor>
}
""".strip()
        },
        {
            "role": "user",
             "content": """
ADAM: (calling out) Hello?
ADAM: (calling out) The progeny returns!
ADAM: Hellooooo...
REBECCA HAYES: (from the kitchen) In the kitchen!
REBECCA HAYES: Adam, sweetie, what are you doing here!
ADAM: (into the hug) Yeah, good to see you too, Mom. Where's Dad?
REBECCA HAYES: Grocery shopping.
REBECCA HAYES: Did we know you were coming this weekend?
ADAM: Nope, it was sort of a last minute thing...(seeing the kitchen) Are you...baking?
REBECCA HAYES: Um...yes. Is there a chance you can talk to Dr. Bright and leave it at that? Let her and The AM take it from here?
ADAM: Sure, Mom. We can try that.
REBECCA HAYES: Mmhm. Sure.
REBECCA HAYES: You should get going.
ADAM: Yeah.(then, playfully conspiratorial) But first, tell me everything you know about alchemy.
FADE TO:
DR. BRIGHT: Hmm, the staff of Caduceus...and she said it was the symbol for salt?
ADAM: Yeah. But it can also mean base? Or body? And this little curved bit might have to do with mercury, or the mind.
CALEB: Did she recognize the arrow type bits?
ADAM: No. She said it doesn't match the other symbols, like it was from a different discipline entirely.
DR. BRIGHT: (peering closer) Hm, I think she might be right. It's not an alchemical or medical symbol, but it does look familiar.
CALEB: Since when does everyone know what alchemical symbols look like?
ADAM: (to Dr. Bright) Do you know where it's from?
DR. BRIGHT: (to Adam) Not off the top of my head. (to Caleb) Alchemy and Atypical science have gone hand in hand for centuries. A lot of the older documents we have about Atypicals use alchemical symbols. I may not be a medical doctor, but most people who work with Atypicals read all the scholarship there is, given that there isn't much. I imagine Rebecca is familiar for the same reason.
ADAM: (muttering) Something like that...
CALEB: Older documents? Like, ancient texts about Atypicals?
DR. BRIGHT: Nothing quite so exciting as "ancient texts". It's the same as any other scientific discipline - there's years and years of research and discoveries. Given Oliver Ritz's ability, he was quite interested in the study and lent me some of his books after he was released.
CALEB: Oliver Ritz- that's the transmutation guy right? Mark's mentioned him a few times.
DR. BRIGHT: That's the one. He often calls himself an alchemist actually- he's always turning something into something else. It was...it was quite helpful a few years ago.
CALEB: Right.
ADAM: He's an alchemist? So he might know something.
DR. BRIGHT: He could. But he's away in Europe at the moment - Mark has been trying to reach him, but Oliver has been..."ghosting" him. Is that the right word?
ADAM: Yeah, Dr. Bright.
CALEB: (grumbling) A bit rich coming from Mark- he's been ghosting me the past week.
ADAM: Really? Is he okay?
DR. BRIGHT: He's fine - I saw him yesterday. He's just been caught up with something - he won't tell me what though, so he's either been fired and is trying to find a new job before I find out or he's seeing someone new.
CALEB: Well, can you tell him to text me back?
ADAM: (mumbling) Doesn't feel good, does it...
DR. BRIGHT: I'll do my best. (then) You know, I'm happy to pass along the material that Oliver gave me.
ADAM: Really?
CALEB: That'd be great, Dr. Bright.
DR. BRIGHT: I don't know how helpful it will be. But Oliver was into some...fringe stuff before he was brought to The AM, so there might be something in there that I missed.
CALEB: Thank you.
DR. BRIGHT: So. Adam-
ADAM: (jumping in) Thank you. For letting us borrow Agent Green's files. I promise we'll get them back to you.
DR. BRIGHT:  It's no problem, Adam. I'm just glad that someone will get some use out of them. They've just been gathering dust in my office.
ADAM: How have you...I mean. How are you? I know it's been a few years but...
DR. BRIGHT: It's funny. Sometimes I feel each and every one of the one thousand one hundred and one days since Owen died. Like they're each a lifetime I've lived without hearing his voice. And other days it feels like yesterday. And I can still hear him. Telling me to be happy. (beat) I'm sorry, I shouldn't have- it's just that sometimes, standing in this lobby, so close to where he- (beat) It never leaves you. Grief isn't linear and it has no end point. (then) I want you both to be careful, Adam. I know that you're capable young men - adults - but if Caleb's friend has really been kidnapped...you need to protect each other. (pointed) Nothing is worth losing someone you love.
MAGS: (distant) What!? That's nuts!
CALEB: (distant) I know!
ADAM: I know. We'll be careful. I promise.
DR. BRIGHT: Okay. (beat) Okay. (beat) We should probably go collect Caleb and get those files - he'll talk to Mags all day if we let him.
ADAM: I...I think I might just hang here, if that's okay? I...I don't know that I'm ready to go deeper into The AM.
DR. BRIGHT: You know, Adam, Ellie- (beat) Your aunt, she...she's been doing better. Since Helen. I won't say she's totally changed her ways, but-
ADAM: You're right, we should get those files and get out of your hair.
DR. BRIGHT: Right. (then) I'll be back.
MAGS: Hey, Adam, long time!
ADAM: Hey, Mags. How are you?
MAGS: I'm doing good- nice to see you both. It's a been a while!
ADAM: Yeah. Yeah, I  guess it has.
MAGS: I thought you two...
ADAM: We did. We just- there's weird stuff going on...we're just helping  each other out.
MAGS: Yeah, Caleb said. Sounds pretty gnarly. You're borrowing some of Owen's old files?
ADAM: Yeah. Did he ever mention anything  to you about any secret Atypical societies or-
ALICE: MagsyMagsyMags!
MAGS: (overlapping) Alice!
ALICE: Adam?
ADAM: Alice.
ALICE: Um, hi.
ADAM: Hi.
ALICE: ...what are you doing here?
ADAM: What are you doing here?
ALICE: Weekly group therapy.
MAGS: Right, speaking of, I should probably go grab the new members-
ALICE: (to Mags) Anyone cool this week?
MAGS: You'll just have to wait and see.
ALICE: Ugh, fine. I'll be in in a sec.
MAGS: Sure. (then, to Adam) Nice to see you, Adam.
ADAM: Yeah, you too, Mags.
ALICE: (refocusing on Adam) So what are you doing here?
ADAM: (faltering) Oh, um, Caleb and I are- we were talking to Dr. Bright about- we need these files-
ALICE: Oh right, the weird Yale thing. You guys are still looking into that?
ADAM: I mean, yeah.  It's still happening. Especially now with Caleb's friend missing-
ALICE: Whoa, what? Someone is missing? Like, legit missing?
ADAM: ...Caleb didn't tell you that?
ALICE: No. No, he didn't. (beat) I'm gonna kill him.
ADAM: Alice-
ALICE: No, if you guys are involved in something, like, actually dangerous, and Caleb didn't tell me-
ADAM: I'm sure he just didn't want to worry you-
ALICE: Screw that, I'm worried.
ADAM: Please don't tell him I told you.
ALICE: Fine. (beat) I'm sure I can get it out of him anyway.
ADAM: I'm kind of surprised he hasn't caught you up. I thought you guys told each other everything.
ALICE: We do, but...I don't know, he's  been weirdly cagey recently. And, anyway, you're probably right, he probably didn't want to worry me. He's always trying to protect me.
ADAM: (dry) What a terrible older brother he is.
""".strip()
         },
        {
            "role": "assistant",
             "content": """
{
    "introducer": "ADAM",
    "counsellor": "ADAM",
    "counselee": "DR. BRIGHT"
}
""".strip()
         },
        {
            "role": "user", 
            "content": transcript.strip()
        },
    ]

    sleep(10)
    
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        max_tokens=1024,
        temperature=0.1,
    )
    
    output = json.loads(r["choices"][0]["message"]["content"])
    
    return idx, output

In [65]:
pool =  ThreadPoolExecutor(max_workers=5)
fs = [
    pool.submit(run_identifier, row.Index, row.transcript) 
    for row in df.itertuples()
    if not row.identifiers
]

In [66]:
len(fs)

10

In [67]:
it = as_completed(fs)
for _ in range(len(fs)):
    try:
        idx, result = next(it).result()
    except Exception as e:
        print(idx, repr(e))
        continue
    df['identifiers'].loc[idx] = json.dumps(result)    
    print(idx, result)   

0 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
2 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
3 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
4 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
1 {'introducer': 'Speaker 0', 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
7 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
6 {'introducer': 'Speaker 0', 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
9 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
8 {'introducer': None, 'counsellor': 'Speaker 0', 'counselee': 'Speaker 1'}
10 {'introducer': None, 'counsellor': 'Speaker 1', 'counselee': 'Speaker 2'}


In [69]:
# print(df.loc[1].transcript)

In [70]:
def as_sharegpt(row):
    try:
        ids = json.loads(row.identifiers)
        doctor = ids['counsellor']
        patient = ids['counselee']
        introducer = ids['introducer']
        others = ids.get('others', [])
        assert patient and doctor
    except Exception as e:
        # df.identifiers.loc[row.Index] = None
        print(row.Index, repr(e))
        return None

    pat = r"^\s*(" + re.escape(doctor) + r'|' + re.escape(patient) + r"|.*)\:\s*"
    splits = re.split( pat, row.transcript, flags=re.M, )

    convo = [{"from": "system", "value": row.title}]

    last_role = None
    
    for name, content in zip(splits[1::2], splits[2::2]):
        # print([name, content])
        if name == introducer and len(convo) <= 1:
            convo[0]["value"] = (convo[0]["value"] + "\n" + content.strip()).strip()
            continue
        
        if name == patient:
            role = "human"
        elif name == doctor:
            role = "gpt"
        else:
            continue
        
        if role == last_role:
            convo[-1]["value"] = (convo[-1]["value"] + "\n" + content.strip()).strip()
        else:            
            convo.append({"from": role, "value": content.strip()})
        last_role = role        

    if len(convo) < 3:
        # df.identifiers.loc[row.Index] = None
        print(row.Index, convo)
        return None
    
    # if convo[1]["from"] == "gpt":
    #     convo.pop(1)
    
    return {"conversations": convo}

In [71]:
with open(outfile, "w") as f:
    for row in df.itertuples():
        data = as_sharegpt(row)
        if not data:
            continue
        f.write(json.dumps(data) + "\n")

In [95]:
!head {outfile}

{"conversations": [{"from": "human", "value": "This sub is depressing as fuck. I get it's a mental health sub but every post that pops up in my feed is \"I'm going to kill mysef\" \nIs this a sub for people to post that they're going to kill themselves and this is their last place to go to? It's crazy the amount of posts I see like this and I get it, the world is in a very dark time right now but god damn I thought this was a place had more variety to it...."}, {"from": "gpt", "value": "Seconded I'm a very sensitive person and reading a couple of posts titles similar to this in one setting is just not for me..\nI might just leave this community as well lol"}]}
{"conversations": [{"from": "human", "value": "This sub is depressing as fuck. I get it's a mental health sub but every post that pops up in my feed is \"I'm going to kill mysef\" \nIs this a sub for people to post that they're going to kill themselves and this is their last place to go to? It's crazy the amount of posts I see li

In [96]:
!ls *.jsonl

reddit.jsonl  video_transcripts.jsonl  web_content.jsonl


In [100]:
import glob

ds = []
for fname in glob.glob("*.jsonl"):
    with open(fname) as f:
        ds.extend(json.loads(l) for l in f.readlines()) 

with open("convos.json", 'w') as f:
    json.dump(ds, f)

In [101]:
dataset = load_dataset(".",  data_files=["convos.json"])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1189.54it/s]
Generating train split: 1573 examples [00:00, 5845.86 examples/s]


In [102]:
len(dataset['train'])

1573

In [119]:
dataset.push_to_hub("devxpy/therapychat")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  8.94it/s]


In [118]:
dataset['train']["conversations"][:10]

[[{'from': 'human',
   'value': 'This sub is depressing as fuck. I get it\'s a mental health sub but every post that pops up in my feed is "I\'m going to kill mysef" \nIs this a sub for people to post that they\'re going to kill themselves and this is their last place to go to? It\'s crazy the amount of posts I see like this and I get it, the world is in a very dark time right now but god damn I thought this was a place had more variety to it....'},
  {'from': 'gpt',
   'value': "Seconded I'm a very sensitive person and reading a couple of posts titles similar to this in one setting is just not for me..\nI might just leave this community as well lol"}],
 [{'from': 'human',
   'value': 'This sub is depressing as fuck. I get it\'s a mental health sub but every post that pops up in my feed is "I\'m going to kill mysef" \nIs this a sub for people to post that they\'re going to kill themselves and this is their last place to go to? It\'s crazy the amount of posts I see like this and I get i